<h1>Text data demo (tensorflow version)</h1>
Roughly the same as the other RNN notebook, implemented in tensorflow.

In [1]:
from ntb.datasets import textdata
import matplotlib.pyplot as pp
import numpy as np
import tensorflow as tf
import tensorflow.contrib
%matplotlib inline
%reload_ext autoreload

In [2]:
seq_length = 40

In [3]:
text_data,encode,decode = textdata.load(seq_length=seq_length,filename='us_constitution.txt',val_frac=.01,stride=1)

In [4]:
vocab_length = len(text_data['charmap'])
embedding_dim = 32
hidden_dim = 72
batch_size = 50
lstm_layers = 2
num_train = len(text_data['X_train'])

In [5]:
mask_1 = np.repeat(a=(np.arange(seq_length).reshape(1,-1)).astype(np.float32),axis=0,repeats=batch_size)*seq_length/(np.arange(seq_length).sum())
mask_2 = np.repeat(a=np.ones([1,seq_length]).astype(np.float32),axis=0,repeats=batch_size)

In [6]:
g = tf.Graph()
sess = tf.InteractiveSession(graph=g,config=tf.ConfigProto(log_device_placement=True))

In [7]:
with g.as_default():
    with tf.variable_scope('input_layer'):
        x = tf.placeholder(dtype=tf.int32,shape=[None,seq_length],name="x")
        y = tf.placeholder(dtype=tf.int32,shape=[None,seq_length],name="y")
        lr = tf.placeholder(dtype=tf.float32,shape=[],name="learning_rate")
        loss_mask = tf.placeholder(dtype=tf.float32,shape=[None,seq_length],name="loss_mask")
        temp = tf.placeholder(dtype=tf.float32,name="temperature")
        keep_prob = tf.placeholder(dtype=tf.float32,name="keep_prob")
    if(embedding_dim is not None):
        with tf.variable_scope('embedding_layer'):
            w_emb = tf.get_variable(dtype=tf.float32,shape=[vocab_length,embedding_dim],name="embedding_weights")
        def embed(x):
            return tf.nn.embedding_lookup(w_emb,x)
    else:
        def embed(x):
            return tf.one_hot(depth=vocab_length,indices=x)
    with tf.variable_scope('embedding_layer'):
        x_emb = embed(x)
    with tf.variable_scope('lstm_layer') as lstm_scope:
        stacked_lstm = tf.contrib.rnn.MultiRNNCell(cells=[tf.contrib.rnn.LayerNormBasicLSTMCell(hidden_dim,dropout_keep_prob=keep_prob) for _ in range(lstm_layers)])
        drnn_out,drnn_final_state = tf.nn.dynamic_rnn(cell=stacked_lstm,inputs=x_emb,dtype=tf.float32,scope="drnn")
    with tf.variable_scope('output_layer'):
        w_out = tf.get_variable(dtype=tf.float32,shape=[hidden_dim,vocab_length],name="output_weights")
        b_out = tf.get_variable(dtype=tf.float32,shape=[vocab_length],name="output_biases")
        scores = tf.tensordot(drnn_out,w_out,axes=([2],[0]))
    with tf.variable_scope('loss'):
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=scores)*loss_mask)
        optim = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
    with tf.variable_scope(lstm_scope) as scope:
        counter = tf.constant(value=0,dtype=tf.int32)
        sample_length = tf.placeholder(dtype=tf.int32,shape=[],name="sample_length")
        x_sample_primer = tf.placeholder(dtype=tf.int64,shape=[1,None],name="x_sample_primer")
        x_sample_primer_emb = embed(x_sample_primer)
        scope.reuse_variables()
        primer_drnn_out,primer_drnn_state = tf.nn.dynamic_rnn(cell=stacked_lstm,inputs=x_sample_primer_emb,dtype=tf.float32,scope="drnn")
        primer_scores = tf.matmul(primer_drnn_out[:,-1,:],w_out)
        sample = tf.multinomial(primer_scores/temp,1)[0]
        samples = tf.TensorArray(dtype=tf.int64,size=sample_length,element_shape=[1],clear_after_read=False)
        samples = samples.write(0,sample)
        state = primer_drnn_state

        def body_(counter,samples,state):
            last_sample = samples.read(counter)
            last_sample_emb = embed(last_sample)
            scope.reuse_variables()
            h,new_state = stacked_lstm(inputs=last_sample_emb,state=state,scope="drnn/multi_rnn_cell")
            new_scores = tf.matmul(h,w_out)
            new_sample = tf.multinomial(new_scores/temp,1)[0]
            samples = samples.write(counter+1,new_sample)
            return counter+1,samples,new_state
        _,sample_out,_ = tf.while_loop(cond=lambda t,*_:t<sample_length-1,body=body_,loop_vars=[counter,samples,state])
        sample_out = sample_out.concat()
    init = tf.global_variables_initializer()

In [8]:
sess.run(init)

In [9]:
def get_batch():
    idx = np.random.choice(num_train,batch_size)
    return text_data['X_train'][idx],text_data['y_train'][idx]
def print_sample(length,temperature=1.,primer=" "):
    r, = sess.run([sample_out],feed_dict={x_sample_primer:np.array([encode(primer)]),sample_length:length,temp:temperature,keep_prob:1.})
    print(decode(r))

In [10]:
iterations = 15000
print_every = 1000
def train(iterations=iterations,print_every=print_every,learning_rate=3e-4):
    loss_avg = 0
    for i in range(iterations+1):
        X_batch,y_batch = get_batch()
        loss_,_ = sess.run([loss,optim],feed_dict={x:X_batch,y:y_batch,lr:learning_rate,keep_prob:1.,loss_mask:mask_2})
        loss_avg += loss_
        if i == 0:
            loss_avg *= print_every
        if (i % print_every) == 0:
            loss_avg /= print_every
            print("Iteration ({}/{})".format(i,iterations),"Loss:",loss_avg,"\n")
            print("Sample:\n----------------------------------------")
            print_sample(length=200,temperature=.7,primer=" ")
            print("----------------------------------------\n")
            loss_avg = 0

In [11]:
train(learning_rate=1e-2,print_every=1000)

Iteration (0/15000) Loss: 4.26951026917 

Sample:
----------------------------------------
IdL Kal3ov3)zDdeykfdxe7-vNeabBUDauCV4qB EBe,Sz1tLhn dOTDoexz7cFaBeBnjOs-6dyPh5Q4I,se0bfLpgie,6eo Nzyho-pokbeSl1bc1
.GC5Qd,3Lh3lFzbn8QI,1B:ehLp19IkJiD3oe5AL:Se5L3 eacBhoaOV5Nl1Yu
qdphModrlSKraS":j.suaq
----------------------------------------

Iteration (1000/15000) Loss: 1.21804626286 

Sample:
----------------------------------------
States shall be
so constitution; but the Constitution by approved by Representatives in which of them
this Concurrence of the United States shall be entitled in the Senate, shall issue.

Amendment 21

----------------------------------------

Iteration (2000/15000) Loss: 0.644254736125 

Sample:
----------------------------------------
from the United States, during their Attendance of President of the United States, shall be appointed to be searched, which shall not be suspended, unless the accused to any person who has held wall o
---------------------------------

In [43]:
print_sample(length=1000,primer='The se',temperature=.5)

cond Class at the Expiration of the fourth Year, and of the term of the President, the votes shall be taken by
states, the House of Representatives shall
immediately choose by Ballot the Vice-President, or hold any office, civil or military, under the
United States, shall be a Representative who shall not have attained to the Age of thirty
Years, and been nine Years a Citizen of the United States, and who
shall not, when elected, be an Inhabitant of that State for which he shall be delivered up on Claim
of the Peesty in such State, being Citizens of age placanf those presented twice shall be removed from Arrest during the Obligation of Contracts, or grant any
Title of Nobility.

No State shall, during the Time for which he was elected, as follows:

Each State shall
in all Cases whatsoever, over such District
(not exceeding three on the list of those voted for as
President, and such person shall act according to the discipline
prescribed by Congress;

To exercise exclusive Legislation i

In [11]:
saver = tf.train.Saver()

In [16]:
saver.save(sess,'./tf_textdata_model')

'./tf_textdata_model'

In [13]:
saver.restore(sess,'./tf_textdata_model')

INFO:tensorflow:Restoring parameters from ./tf_textdata_model
